d
# Distributed Inference with mapInPandas

Train sklearn model and log with MLflow.

In [0]:
ls /dbfs/databricks-datasets/learning-spark-v2/sf-airbnb

In [0]:
import mlflow.sklearn
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

df = pd.read_csv("/dbfs/databricks-datasets/learning-spark-v2/sf-airbnb/sf-airbnb-numeric.csv").drop(["zipcode"], axis=1)
df.head()

--------------------------------------------------------------------------- 
 FileNotFoundError Traceback (most recent call last)
 <command-4360215274556447> in <module> 
 5 from sklearn . model_selection import train_test_split
 6 
 ----> 7 df = pd . read_csv ( "/dbfs/databricks-datasets/learning-spark-v2/sf-airbnb/sf-airbnb-numeric.csv" ) . drop ( [ "zipcode" ] , axis = 1 ) 
 8 df . head ( ) 

 /databricks/python/lib/python3.7/site-packages/pandas/io/parsers.py in parser_f (filepath_or_buffer, sep, delimiter, header, names, index_col, usecols, squeeze, prefix, mangle_dupe_cols, dtype, engine, converters, true_values, false_values, skipinitialspace, skiprows, skipfooter, nrows, na_values, keep_default_na, na_filter, verbose, skip_blank_lines, parse_dates, infer_datetime_format, keep_date_col, date_parser, dayfirst, cache_dates, iterator, chunksize, compression, thousands, decimal, lineterminator, quotechar, quoting, doublequote, escapechar, comment, encoding, dialect, error_bad_lines, warn_bad_lines, delim_whitespace, low_memory, memory_map, float_precision) 
 674 )
 675 
 --> 676 return _read ( filepath_or_buffer , kwds ) 
 677 
 678 parser_f . __name__ = name

 /databricks/python/lib/python3.7/site-packages/pandas/io/parsers.py in _read (filepath_or_buffer, kwds) 
 446 
 447 # Create the parser. 
 --> 448 parser = TextFileReader ( fp_or_buf , ** kwds ) 
 449 
 450 if chunksize or iterator : 

 /databricks/python/lib/python3.7/site-packages/pandas/io/parsers.py in __init__ (self, f, engine, **kwds) 
 878 self . options [ "has_index_names" ] = kwds [ "has_index_names" ] 
 879 
 --> 880 self . _make_engine ( self . engine ) 
 881 
 882 def close ( self ) : 

 /databricks/python/lib/python3.7/site-packages/pandas/io/parsers.py in _make_engine (self, engine) 
 1112 def _make_engine ( self , engine = "c" ) : 
 1113 if engine == "c" : 
 -> 1114 self . _engine = CParserWrapper ( self . f , ** self . options ) 
 1115 else : 
 1116 if engine == "python" : 

 /databricks/python/lib/python3.7/site-packages/pandas/io/parsers.py in __init__ (self, src, **kwds) 
 1889 kwds [ "usecols" ] = self . usecols
 1890 
 -> 1891 self . _reader = parsers . TextReader ( src , ** kwds ) 
 1892 self . unnamed_cols = self . _reader . unnamed_cols
 1893 

 pandas/_libs/parsers.pyx in pandas._libs.parsers.TextReader.__cinit__ () 

 pandas/_libs/parsers.pyx in pandas._libs.parsers.TextReader._setup_parser_source () 

 FileNotFoundError : [Errno 2] File /dbfs/databricks-datasets/learning-spark-v2/sf-airbnb/sf-airbnb-numeric.csv does not exist: '/dbfs/databricks-datasets/learning-spark-v2/sf-airbnb/sf-airbnb-numeric.csv'

In [0]:
import mlflow.sklearn
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

with mlflow.start_run(run_name="sklearn-rf-model") as run:
  # Import the data
  df = pd.read_csv("/dbfs/databricks-datasets/learning-spark-v2/sf-airbnb/sf-airbnb-numeric.csv").drop(["zipcode"], axis=1)
  X_train, X_test, y_train, y_test = train_test_split(df.drop(["price"], axis=1), df[["price"]].values.ravel(), random_state=42)

  # Create model, train it, and create predictions
  rf = RandomForestRegressor(n_estimators=100, max_depth=10)
  rf.fit(X_train, y_train)
  predictions = rf.predict(X_test)

  # Log model
  mlflow.sklearn.log_model(rf, "random-forest-model")

  # Log params
  mlflow.log_param("n_estimators", 100)
  mlflow.log_param("max_depth", 10)

  # Log metrics
  mlflow.log_metric("mse", mean_squared_error(y_test, predictions))
  mlflow.log_metric("mae", mean_absolute_error(y_test, predictions))  
  mlflow.log_metric("r2", r2_score(y_test, predictions))  

Create Spark DataFrame and apply model to Spark DataFrame.

In [0]:
sparkDF = spark.createDataFrame(X_train)

Apply the model in parallel with mapInPandas

In [0]:
import mlflow.sklearn
import pandas as pd

def predict(iterator):
  model_path = f"runs:/{run.info.run_uuid}/random-forest-model" # Load model
  model = mlflow.sklearn.load_model(model_path)
  for features in iterator:
    yield pd.concat([features, pd.Series(model.predict(features), name="prediction")], axis=1)
    
display(sparkDF.mapInPandas(predict, """`host_total_listings_count` DOUBLE,`neighbourhood_cleansed` BIGINT,`latitude` DOUBLE,`longitude` DOUBLE,`property_type` BIGINT,`room_type` BIGINT,`accommodates` DOUBLE,`bathrooms` DOUBLE,`bedrooms` DOUBLE,`beds` DOUBLE,`bed_type` BIGINT,`minimum_nights` DOUBLE,`number_of_reviews` DOUBLE,`review_scores_rating` DOUBLE,`review_scores_accuracy` DOUBLE,`review_scores_cleanliness` DOUBLE,`review_scores_checkin` DOUBLE,`review_scores_communication` DOUBLE,`review_scores_location` DOUBLE,`review_scores_value` DOUBLE, prediction double"""))
